In [1]:
# === XAUUSD PROFIT-ONLY TRADING BOT (MT5 + Python) ===

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import time

# === CONFIG ===
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.1
MIN_PROFIT_USD = 1  # close trade when profit >= $1

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

symbol_info = mt5.symbol_info(SYMBOL)
if symbol_info is None:
    raise RuntimeError(f"Symbol {SYMBOL} not found")
if not symbol_info.visible:
    mt5.symbol_select(SYMBOL, True)

# === HELPER: Get recent candles ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None:
        raise RuntimeError(f"Failed to retrieve market data: {mt5.last_error()}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === HELPER: Entry Signal (Engulfing + RSI < 40 Buy or RSI > 60 Sell) ===
def get_trade_signal(df):
    close = df['close'].astype(float)
    open_ = df['open'].astype(float)
    high = df['high'].astype(float)
    low = df['low'].astype(float)

    rsi = talib.RSI(close, timeperiod=14)[-1]
    engulf = np.array(talib.CDLENGULFING(open_, high, low, close))[-1]

    if engulf > 0 and rsi < 40:
        return "buy"
    elif engulf < 0 and rsi > 60:
        return "sell"
    return None

# === HELPER: Open Trade ===
def open_trade(direction):
    tick = mt5.symbol_info_tick(SYMBOL)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "profit_only_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Opened {direction.upper()} at {price}")
    else:
        print(f"Order failed: {result.retcode}")

# === HELPER: Close Trade if profit >= $1 ===
def check_and_close():
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions:
        pos = positions[0]
        profit = pos.profit
        if profit >= MIN_PROFIT_USD:
            close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
            price = mt5.symbol_info_tick(SYMBOL).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).ask
            close_req = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": SYMBOL,
                "volume": pos.volume,
                "type": close_type,
                "position": pos.ticket,
                "price": price,
                "deviation": 20,
                "magic": 10001,
                "comment": "close_in_profit",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_req)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"Trade closed in profit: ${profit:.2f}")
            else:
                print(f"Close failed: {result.retcode}")

# === MAIN LOOP ===
print("Running XAUUSD profit-only bot...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        if not mt5.positions_total():
            df = get_data(SYMBOL, TIMEFRAME, 100)
            signal = get_trade_signal(df)
            if signal:
                open_trade(signal)

        time.sleep(60)  # wait 1 min before checking again
except KeyboardInterrupt:
    print("Stopped by user")
finally:
    mt5.shutdown()


Running XAUUSD profit-only bot...


KeyError: -1

In [2]:
# === XAUUSD PROFIT-ONLY TRADING BOT (MT5 + Python) ===

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import time

# === CONFIG ===
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.1
MIN_PROFIT_USD = 1  # close trade when profit >= $1

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

symbol_info = mt5.symbol_info(SYMBOL)
if symbol_info is None:
    raise RuntimeError(f"Symbol {SYMBOL} not found")
if not symbol_info.visible:
    mt5.symbol_select(SYMBOL, True)

# === HELPER: Get recent candles ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to retrieve market data: {mt5.last_error()}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === HELPER: Entry Signal (Engulfing + RSI < 40 Buy or RSI > 60 Sell) ===
def get_trade_signal(df):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    if len(rsi) == 0:
        return None

    engulf = talib.CDLENGULFING(open_, high, low, close)
    if len(engulf) == 0:
        return None

    if engulf[-1] > 0 and rsi[-1] < 40:
        return "buy"
    elif engulf[-1] < 0 and rsi[-1] > 60:
        return "sell"
    return None

# === HELPER: Open Trade ===
def open_trade(direction):
    tick = mt5.symbol_info_tick(SYMBOL)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "profit_only_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Opened {direction.upper()} at {price}")
    else:
        print(f"Order failed: {result.retcode}")

# === HELPER: Close Trade if profit >= $1 ===
def check_and_close():
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions:
        pos = positions[0]
        profit = pos.profit
        if profit >= MIN_PROFIT_USD:
            close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
            price = mt5.symbol_info_tick(SYMBOL).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).ask
            close_req = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": SYMBOL,
                "volume": pos.volume,
                "type": close_type,
                "position": pos.ticket,
                "price": price,
                "deviation": 20,
                "magic": 10001,
                "comment": "close_in_profit",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_req)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"Trade closed in profit: ${profit:.2f}")
            else:
                print(f"Close failed: {result.retcode}")

# === MAIN LOOP ===
print("Running XAUUSD profit-only bot...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        if not mt5.positions_total():
            df = get_data(SYMBOL, TIMEFRAME, 100)
            signal = get_trade_signal(df)
            if signal:
                open_trade(signal)

        time.sleep(60)  # wait 1 min before checking again
except KeyboardInterrupt:
    print("Stopped by user")
finally:
    mt5.shutdown()

Running XAUUSD profit-only bot...


RuntimeError: Failed to retrieve market data: (-10001, 'IPC send failed')